# W207 Final Project: Facial Recognition

## Introduction

This is the final project submission from Team MapReduce,MapReuse,MapRecycle for W207: Applied Machine Learning as part of the MIDS program at UC Berkeley. We are working with data from https://www.kaggle.com/c/facial-keypoints-detection/data and the goal is to accurately identify facial features on an (x, y) coordinate system based on an input image. 

## Communicate on SLACK when working on main notebook as much as possible

*Data Observations*
- There are half face images, etc

*Ideas to try*
- Use other features to predict one specific feature

- Break out the image into its own matrix where each pixel is a feature and then predict each facial characteristic position
    - First use subset of images that contain all key points and then just predict the locations. 

*Feature Improvement Ideas*
- Rotating the images a bit
- Blurring the images a bit
- Maybe normally the pixel values by average pixel shade of every picture
- Seperate the image into different quadrants 
- Compressing 256 base image to 16 base image. Just black and white? Interval of -1 to 1?

*Knockout List/Plan*
- Print first 10-20 images generally. 
- Print edge cases for different facial feature locations.

- Very basic test with a couple different basic algorithms. 
- Start with very basic one hidden layer perception
- Improve neural network to be CNN architecture. 
- Prepare a summary table of quality metrics: L2 distance, L1 distance

*Naming/Working Rules*
- train_data, train_labels
- dev_data, dev_labels

- Feature Engineering: put "_transformation done". Ex: train_data_blur

*Useful Items*
- Learn Keras
- https://www.kaggle.com/c/facial-keypoints-detection#getting-started-with-r
- http://adventuresinmachinelearning.com/keras-tutorial-cnn-11-lines/
- https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm#k-NN_regression

## Data

In [1]:
# Import a bunch of libraries.
import time
import os
# os.environ["MKL_THREADING_LAYER"] = "GNU"
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import decimal
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from scipy import stats


from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.models import Sequential
from keras import optimizers
from keras import backend as K

# Set the randomizer seed so results are the same each time.
np.random.seed(0)

#print('theano version:', theano.__version__)
#print('theano config device:', theano.config.device, theano.config.floatX
print('keras version:', keras.__version__)
print('pandas version:', pd.__version__)
print('numpy version:', np.__version__) 

Using TensorFlow backend.


keras version: 2.2.4
pandas version: 0.23.4
numpy version: 1.15.4


In [40]:
# Load the data from downloaded file and split the train into a smaller train and dev

train = pd.read_csv('training.csv')
test = pd.read_csv('test.csv')

print("Columns available:")
print( train.columns.values )
print(train)
def process_datasets(data):
    facial_point_labels = data.iloc[:, :-1]
    images_pixel_feature = data.Image.apply(lambda im: np.fromstring(im, sep=' '))
    
    # Turn below on to the regressions below.
    #images_pixel_feature = pd.DataFrame(images_pixel_feature.values.tolist())
    
    # Filling NAs with 0 
    facial_point_labels = facial_point_labels.fillna(0)
    images_pixel_feature = images_pixel_feature.fillna(0)
    
    return images_pixel_feature, facial_point_labels

train_all_features, train_all_labels = process_datasets(train)

percent_to_dev = 0.2 # 20% of the data will go to dev

train_data, dev_data, train_labels, dev_labels = train_test_split(train_all_features, train_all_labels, test_size=percent_to_dev)
test_data, test_labels = process_datasets(test)

print( "\nTrain data shape", train_data.shape)
print( "Dev data shape", dev_data.shape)
print( "Train labels shape", train_labels.shape)
print( "Dev labels shape", dev_labels.shape)

print( "\nData Generation Finished")


Columns available:
['left_eye_center_x' 'left_eye_center_y' 'right_eye_center_x'
 'right_eye_center_y' 'left_eye_inner_corner_x' 'left_eye_inner_corner_y'
 'left_eye_outer_corner_x' 'left_eye_outer_corner_y'
 'right_eye_inner_corner_x' 'right_eye_inner_corner_y'
 'right_eye_outer_corner_x' 'right_eye_outer_corner_y'
 'left_eyebrow_inner_end_x' 'left_eyebrow_inner_end_y'
 'left_eyebrow_outer_end_x' 'left_eyebrow_outer_end_y'
 'right_eyebrow_inner_end_x' 'right_eyebrow_inner_end_y'
 'right_eyebrow_outer_end_x' 'right_eyebrow_outer_end_y' 'nose_tip_x'
 'nose_tip_y' 'mouth_left_corner_x' 'mouth_left_corner_y'
 'mouth_right_corner_x' 'mouth_right_corner_y' 'mouth_center_top_lip_x'
 'mouth_center_top_lip_y' 'mouth_center_bottom_lip_x'
 'mouth_center_bottom_lip_y' 'Image']
      left_eye_center_x  left_eye_center_y  right_eye_center_x  \
0             66.033564          39.002274           30.227008   
1             64.332936          34.970077           29.949277   
2             65.057053  


Train data shape (5639,)
Dev data shape (1410,)
Train labels shape (5639, 30)
Dev labels shape (1410, 30)

Data Generation Finished


In [32]:
def mean_squared_error_with_missing(y_true, y_pred):
    # set constants
    zeroes = tf.constant([0.])
    maxWidth = tf.constant([96.])
    
    # get the distances for points that are present
    predictedPresent = tf.math.greater_equal(y_pred,zeroes)
    truePresent = tf.math.greater_equal(y_true,zeroes)
    truePositives = tf.logical_and(predictedPresent,truePresent)
    
    # get the classification loss
    classLoss = tf.math.reduce_mean(tf.math.squared_difference(tf.to_float(predictedPresent),tf.to_float(truePresent)))
    
    # get distance loss
    distanceLoss = tf.math.reduce_mean(tf.math.squared_difference(y_true,y_pred)*tf.to_float(truePositives))
    
    #return classLoss + distanceLoss
    return distanceLoss


In [33]:
# Train the model
img_x = 96
img_y = 96
trainx = np.concatenate(train_data.values).reshape(train_data.size,img_x, img_y,1)
trainy = train_labels.values.reshape(train_data.size,30)
print(trainy)

sgd = optimizers.SGD(lr=0.0001)

input_shape = (img_x, img_y,1)
model = keras.Sequential()
model.add(AveragePooling2D(pool_size=(3, 3), strides=(3, 3)))
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu'))
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Flatten())
model.add(Dense(60, activation='linear'))
model.compile(loss=mean_squared_error_with_missing, optimizer=sgd)
model.fit(trainx, trainy)

[[64.86912    33.08325517 30.71359117 ... -1.         49.38855724
  72.89048276]
 [66.5424     38.57655111 31.81489778 ... -1.         49.67475556
  72.97329778]
 [67.42121739 38.47813565 30.64063304 ... -1.         50.39311304
  77.30197565]
 ...
 [68.17427692 40.50944    30.36098462 ... -1.         46.21819077
  77.10294154]
 [65.82554483 32.97494069 24.60038621 ... -1.         47.74984828
  68.80915862]
 [70.51325217 36.40758261 32.08674783 ... -1.         50.41857391
  76.94921739]]
Epoch 1/1
5639/5639 [==============================] - 17s 3ms/step - loss: 251.1871


In [34]:
# Make predictions on the test data
testx = np.concatenate(test_data.values).reshape(test_data.size,img_x, img_y,1)
### PSEUDOCODE ###
pred = model.predict(testx)

In [35]:
print(pred)
print(pred*96)

[[-1.76035345e-01 -1.11105770e-01 -6.01718910e-02 ... -1.22794680e-01
  -1.22983485e-01 -5.05645037e-01]
 [-1.22257024e-01 -5.69638871e-02 -4.49413620e-02 ... -3.42828855e-02
  -1.28223836e-01 -3.92638832e-01]
 [-6.89939320e-01 -4.01766568e-01 -3.12658191e-01 ... -1.47727132e-01
  -4.67063576e-01 -1.54750586e+00]
 ...
 [-5.35804825e-03  1.31239137e-03  2.19286256e-03 ...  4.62329295e-03
  -2.50695180e-03 -1.30405156e-02]
 [-3.35606009e-01 -1.44806564e-01 -1.69048250e-01 ... -2.47371718e-02
  -2.15881258e-01 -7.99439907e-01]
 [-1.58484444e-01 -9.85991657e-02 -8.80832225e-02 ... -8.00112411e-02
  -1.35461047e-01 -4.96528685e-01]]
[[-1.6899393e+01 -1.0666154e+01 -5.7765017e+00 ... -1.1788289e+01
  -1.1806415e+01 -4.8541924e+01]
 [-1.1736674e+01 -5.4685330e+00 -4.3143706e+00 ... -3.2911570e+00
  -1.2309488e+01 -3.7693329e+01]
 [-6.6234177e+01 -3.8569592e+01 -3.0015186e+01 ... -1.4181805e+01
  -4.4838104e+01 -1.4856056e+02]
 ...
 [-5.1437265e-01  1.2598957e-01  2.1051481e-01 ...  4.4383612e

In [36]:
lookup = pd.read_csv('IdLookupTable.csv')
print(lookup)

       RowId  ImageId                FeatureName  Location
0          1        1          left_eye_center_x       NaN
1          2        1          left_eye_center_y       NaN
2          3        1         right_eye_center_x       NaN
3          4        1         right_eye_center_y       NaN
4          5        1    left_eye_inner_corner_x       NaN
5          6        1    left_eye_inner_corner_y       NaN
6          7        1    left_eye_outer_corner_x       NaN
7          8        1    left_eye_outer_corner_y       NaN
8          9        1   right_eye_inner_corner_x       NaN
9         10        1   right_eye_inner_corner_y       NaN
10        11        1   right_eye_outer_corner_x       NaN
11        12        1   right_eye_outer_corner_y       NaN
12        13        1   left_eyebrow_inner_end_x       NaN
13        14        1   left_eyebrow_inner_end_y       NaN
14        15        1   left_eyebrow_outer_end_x       NaN
15        16        1   left_eyebrow_outer_end_y       N

In [37]:
for i in range(len(lookup)):
    featureName = lookup.FeatureName[i]
    featureNum = np.minimum(np.where(train.columns.values == featureName),29)
    imageNum = lookup.ImageId[i]-1
    lookup.Location[i] =  pred[imageNum,featureNum]*96
    if i%100 == 0: print(i)

C:\Users\Maddie\Anaconda3\envs\tensorflowgpu\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


0
100
200
300
400
500
600
700
800
900


KeyboardInterrupt: 

In [38]:
print(lookup)

       RowId  ImageId                FeatureName   Location
0          1        1          left_eye_center_x -16.899393
1          2        1          left_eye_center_y -10.666154
2          3        1         right_eye_center_x  -5.776502
3          4        1         right_eye_center_y  -4.847642
4          5        1    left_eye_inner_corner_x  -4.060133
5          6        1    left_eye_inner_corner_y  -1.625676
6          7        1    left_eye_outer_corner_x  -5.773779
7          8        1    left_eye_outer_corner_y  -2.598828
8          9        1   right_eye_inner_corner_x -10.058563
9         10        1   right_eye_inner_corner_y  -0.212716
10        11        1   right_eye_outer_corner_x   0.570388
11        12        1   right_eye_outer_corner_y  -3.305246
12        13        1   left_eyebrow_inner_end_x  -3.118590
13        14        1   left_eyebrow_inner_end_y  -1.010219
14        15        1   left_eyebrow_outer_end_x -12.758480
15        16        1   left_eyebrow_out